<a href="https://colab.research.google.com/github/johnathanacortesd/Zero_Entreno_Roberta_Mpnet/blob/main/Zero_Entreno_Tono_Tema_Sin_API_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 📊 Interfaz para Concatenar Columnas (Ejecutar esta celda)
# --- Importaciones necesarias ---
import pandas as pd
import ipywidgets as widgets
from google.colab import files
from IPython.display import display, clear_output
import io

# --- Instalación silenciosa de la dependencia para .xlsx ---
!pip install -q openpyxl

# --- Clase principal de la aplicación ---
class VisualConcatenator:
    def __init__(self):
        """Inicializa la interfaz y sus componentes."""
        self.df = None
        self.output_df = None

        # --- Definición de los Widgets (Componentes de la UI) ---
        self.title = widgets.HTML("<h2>🔗 Concatenador Visual de Columnas</h2><hr>")
        self.instructions = widgets.HTML("<h4>Paso 1: Sube tu archivo Excel para empezar.</h4>")

        self.uploader = widgets.FileUpload(
            accept='.xlsx',
            description='Subir Archivo',
            button_style='primary'
        )

        # Contenedor para la segunda parte de la UI (que aparece después de subir)
        self.step2_container = widgets.VBox([])

        # Área de salida para mensajes y resultados
        self.output_area = widgets.Output()

        # --- Observador de eventos ---
        # Llama a la función _handle_upload cuando se sube un archivo
        self.uploader.observe(self._handle_upload, names='value')

    def _handle_upload(self, change):
        """Se activa al subir un archivo. Lee el archivo y crea la UI de selección."""
        # Limpia cualquier contenido previo en el área de salida
        with self.output_area:
            clear_output()

        uploaded_file_info = self.uploader.value
        if not uploaded_file_info:
            return # Si se cancela la subida, no hacer nada

        # Obtener el nombre y contenido del archivo subido
        filename = list(uploaded_file_info.keys())[0]
        content = uploaded_file_info[filename]['content']

        try:
            with self.output_area:
                print(f"⏳ Leyendo '{filename}'...")
                # Leer el archivo Excel desde la memoria
                self.df = pd.read_excel(io.BytesIO(content))
                print(f"✅ ¡Archivo cargado! Se encontraron {self.df.shape[0]} filas.")
                print(f"📊 Columnas disponibles: {', '.join(self.df.columns)}")

            # Si el archivo se leyó bien, crear la siguiente parte de la interfaz
            self._create_column_selector_ui()

        except Exception as e:
            with self.output_area:
                print(f"❌ Error al leer el archivo. Asegúrate de que es un .xlsx válido.")
                print(f"Detalle: {e}")

    def _create_column_selector_ui(self):
        """Crea los widgets para seleccionar columnas y procesar."""

        # Nuevas instrucciones para el usuario
        step2_instructions = widgets.HTML("""
        <h4>Paso 2: Selecciona las columnas a unir.</h4>
        <p><i>Usa <b>Ctrl+Click</b> (o <b>Cmd+Click</b> en Mac) para elegir varias.</i></p>
        """)

        # Widget para seleccionar múltiples columnas
        self.column_selector = widgets.SelectMultiple(
            options=self.df.columns.tolist(),
            description='Columnas:',
            layout=widgets.Layout(height='180px', width='95%'),
            style={'description_width': 'initial'}
        )

        # Botón para iniciar el procesamiento
        self.process_button = widgets.Button(
            description='🔗 Procesar y Descargar Resultado',
            button_style='success',
            icon='cogs',
            layout=widgets.Layout(width='300px', height='40px', margin='10px 0 0 0')
        )

        # Conectar el botón a la función de procesamiento
        self.process_button.on_click(self._process_and_download)

        # Actualizar el contenedor del Paso 2 con los nuevos widgets
        self.step2_container.children = [
            step2_instructions,
            self.column_selector,
            self.process_button
        ]

    def _process_and_download(self, b):
        """Realiza la concatenación, acortado y descarga del archivo."""
        with self.output_area:
            clear_output()

            selected_cols = self.column_selector.value
            if len(selected_cols) < 2:
                print("❌ Error: Debes seleccionar al menos dos columnas para poder unirlas.")
                return

            print("🚀 Procesando los datos...")
            print(f"   - Uniendo columnas: {', '.join(selected_cols)}")

            try:
                # --- Lógica principal ---
                # 1. Función para acortar el texto a 80 palabras
                def acortar_texto(texto, limite=80):
                    palabras = str(texto).split()
                    if len(palabras) > limite:
                        return ' '.join(palabras[:limite]) + '...'
                    return ' '.join(palabras) # .join() limpia espacios múltiples

                # 2. Convertir columnas a texto, rellenar vacíos y unir con un espacio
                texto_unido = self.df[list(selected_cols)].fillna('').astype(str).apply(
                    lambda fila: ' '.join(fila), axis=1
                )

                # 3. Acortar el texto ya unido
                texto_acortado = texto_unido.apply(acortar_texto)

                # 4. Crear el DataFrame final con una única columna llamada "resumen"
                self.output_df = pd.DataFrame({'resumen': texto_acortado})

                print("\n✅ ¡Procesamiento completado con éxito!")

                # --- Lógica de descarga ---
                output_filename = 'resumen_concatenado.xlsx'
                print(f"\n📦 Generando el archivo '{output_filename}' para descarga...")

                # Guardar el DataFrame en un archivo Excel dentro del entorno de Colab
                self.output_df.to_excel(output_filename, index=False, engine='openpyxl')

                # Usar la función de Colab para iniciar la descarga en el navegador
                files.download(output_filename)

            except Exception as e:
                print(f"\n❌ Ocurrió un error inesperado durante el procesamiento.")
                print(f"Detalle: {e}")

    def display_app(self):
        """Muestra la aplicación completa en la celda."""
        display(
            self.title,
            self.instructions,
            self.uploader,
            self.step2_container,
            self.output_area
        )

# --- Punto de entrada: Crear y mostrar la aplicación ---
app = VisualConcatenator()
app.display_app()

HTML(value='<h2>🔗 Concatenador Visual de Columnas</h2><hr>')

HTML(value='<h4>Paso 1: Sube tu archivo Excel para empezar.</h4>')

FileUpload(value={}, accept='.xlsx', button_style='primary', description='Subir Archivo')

VBox()

Output()

In [ ]:
# @title 🔧 Instalaciones

# Optimizado para saturar la GPU y procesar miles de textos por segundo
# Usa batch sizes grandes y procesamiento paralelo

# ===============================================================
# 1) Instalación
# ===============================================================
print("Instalando dependencias...")
!pip install -q transformers sentence-transformers pandas openpyxl tqdm accelerate
print("Listo.")

Instalando dependencias...
Listo.


In [ ]:
# @title 🚀 Analizador de Tono y Tema v13 GPU-OPTIMIZED – Máxima Velocidad

# ===============================================================
# 2) Importaciones
# ===============================================================
import os
import io
import re
import gc
import time

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from sentence_transformers import SentenceTransformer
from google.colab import files

# ===============================================================
# 3) Configuración GPU AGRESIVA
# ===============================================================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    device = torch.device("cuda")
    # Optimizaciones agresivas para GPU
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.cuda.empty_cache()

    # Obtener info de GPU
    gpu_props = torch.cuda.get_device_properties(0)
    total_memory = gpu_props.total_memory / 1024**3
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ VRAM Total: {total_memory:.1f} GB")
    print(f"✓ Compute Capability: {gpu_props.major}.{gpu_props.minor}")
else:
    device = torch.device("cpu")
    print("⚠️  CPU en uso. Active GPU (Runtime > Change runtime type > T4 GPU)")
    raise RuntimeError("GPU requerida para este notebook")

# ===============================================================
# MODELOS - Los más rápidos y precisos
# ===============================================================

# MODELO DE SENTIMIENTO - Rápido y preciso
MODELO_SENTIMIENTO = "cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual"

# MODELO DE EMBEDDINGS - Versión base para balance velocidad/precisión
MODELO_EMBEDDING = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

# PARÁMETROS OPTIMIZADOS PARA GPU T4/V100 (15GB)
# Ajustados para saturar la GPU sin OOM
BATCH_SENT = 2048        # ¡Enorme! Procesará miles de textos a la vez
BATCH_EMB = 4096         # Aún más grande para embeddings
MAX_LEN_SENT = 128       # Balance entre contexto y velocidad
MAX_CHARS_CLEAN = 400

# Agrupamiento
EXACT_PREFIX_LEN = 80
MIN_PREFIX_LEN = 20

# Workers para DataLoader
NUM_WORKERS = 4

# ===============================================================
# 4) Diccionarios para temas
# ===============================================================
TOPIC_KEYWORDS = {
    "Políticas Públicas y Gobierno": ["reforma","decreto","ley","plan nacional","conpes","congreso","gobierno","política"],
    "Gestión Presupuestaria y Recursos": ["presupuesto","licitación","contrato","inversión","vigencias","adición","recursos"],
    "Salud Pública y Sanidad": ["hospital","vacunación","salud pública","epidemia","covid","ips","eps","medicina"],
    "Educación y Desarrollo Académico": ["colegios","universidades","icfes","becas","educación","saber","estudiantes"],
    "Seguridad Ciudadana y Justicia": ["policía","delito","captura","homicidio","fiscalía","juzgado","crimen"],
    "Economía, Empleo y Desarrollo": ["pib","inflación","empleo","desempleo","pymes","exportaciones","economía"],
    "Infraestructura y Obras Públicas": ["vía","doble calzada","intercambiador","vivienda","energía","metro","obra"],
    "Relaciones Exteriores y Cooperación": ["cancillería","acuerdo bilateral","cooperación","embajada","internacional"],
    "Medio Ambiente y Sostenibilidad": ["deforestación","emisiones","renovables","biodiversidad","clima","ambiente"],
    "Transparencia y Participación Ciudadana": ["veeduría","rendición de cuentas","corrupción","participación","transparencia"],
}
TOPIC_LIST = list(TOPIC_KEYWORDS.keys())

# ===============================================================
# 5) Utilidades
# ===============================================================

def sanitize_name(name):
    return re.sub(r"[^A-Za-z0-9_\-]", "_", name)

def clean_text_basic(text):
    if not isinstance(text, str):
        return ""
    t = text.strip().replace("\n", " ")
    t = re.sub(r"[\x00-\x1f\x7f]", " ", t)
    t = re.sub(r"\s+", " ", t)
    if len(t) > MAX_CHARS_CLEAN:
        t = t[:MAX_CHARS_CLEAN].rsplit(" ", 1)[0] if " " in t[:MAX_CHARS_CLEAN] else t[:MAX_CHARS_CLEAN]
    return t

def normalize_for_prefix(text):
    t = text.lower().strip()
    t = re.sub(r"https?://\S+|www\.\S+", " ", t)
    t = re.sub(r"[^\w\sáéíóúüñ]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def print_gpu_usage():
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1024**3
        reserved = torch.cuda.memory_reserved(0) / 1024**3
        print(f"   GPU Memory: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved")

# ===============================================================
# 6) Agrupamiento rápido
# ===============================================================

def agrupar_por_prefijo(textos):
    print("🔍 Agrupando textos similares...")
    grupos = []
    mapa = np.empty(len(textos), dtype=np.int32)
    idx_por_pref = {}

    for i, txt in enumerate(textos):
        pref = normalize_for_prefix(txt)[:EXACT_PREFIX_LEN]
        if len(pref) < MIN_PREFIX_LEN:
            pref = f"_short_{i}"
        gid = idx_por_pref.get(pref)
        if gid is None:
            gid = len(grupos)
            idx_por_pref[pref] = gid
            grupos.append([i])
        else:
            grupos[gid].append(i)
        mapa[i] = gid

    reps = []
    for miembros in grupos:
        rep_idx = max(miembros, key=lambda k: len(textos[k]))
        reps.append(textos[rep_idx])

    print(f"✓ {len(textos)} textos → {len(reps)} grupos ({(1-len(reps)/len(textos))*100:.1f}% reducción)")
    return reps, mapa, grupos

# ===============================================================
# 7) Dataset y Sentimiento GPU-OPTIMIZADO
# ===============================================================

class TextDatasetFast(Dataset):
    def __init__(self, textos, tokenizer, max_len):
        self.textos = textos
        self.tok = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        return self.tok(
            self.textos[idx],
            truncation=True,
            max_length=self.max_len,
            add_special_tokens=True,
            return_tensors=None,
        )

@torch.inference_mode()
def inferir_sentimiento_gpu_fast(reps):
    print(f"\n🎯 Analizando sentimiento con {MODELO_SENTIMIENTO}")
    print(f"   Batch size: {BATCH_SENT} | Max length: {MAX_LEN_SENT}")

    # Cargar modelo
    tokenizer = AutoTokenizer.from_pretrained(MODELO_SENTIMIENTO)
    model = AutoModelForSequenceClassification.from_pretrained(MODELO_SENTIMIENTO)

    # Optimizaciones GPU
    model = model.to(device)
    model.half()  # FP16 para 2x más rápido
    model.eval()

    print_gpu_usage()

    # Dataset
    dataset = TextDatasetFast(reps, tokenizer, MAX_LEN_SENT)
    collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)
    dataloader = DataLoader(
        dataset,
        batch_size=BATCH_SENT,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        collate_fn=collator,
        prefetch_factor=2
    )

    # Inferencia
    all_preds = []
    t_start = time.time()

    with torch.amp.autocast(device_type="cuda", dtype=torch.float16):  # Mixed precision
        for batch in tqdm(dataloader, desc="Sentimiento"):
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
            logits = model(**batch).logits
            preds = logits.argmax(dim=-1).cpu().numpy()
            all_preds.extend(preds)

    t_elapsed = time.time() - t_start
    print(f"   ⚡ Procesados {len(reps)} textos en {t_elapsed:.2f}s ({len(reps)/t_elapsed:.0f} textos/s)")
    print_gpu_usage()

    # Mapear labels (XLM-RoBERTa: 0=neg, 1=neu, 2=pos)
    label_map = {0: "Negativo", 1: "Neutro", 2: "Positivo"}
    sentimientos = [label_map[int(p)] for p in all_preds]

    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()

    return sentimientos

# ===============================================================
# 8) Temas GPU-OPTIMIZADO
# ===============================================================

def inferir_temas_gpu_fast(reps):
    print(f"\n🏷️  Clasificando temas con {MODELO_EMBEDDING}")
    print(f"   Batch size: {BATCH_EMB}")

    # Cargar encoder con optimizaciones
    encoder = SentenceTransformer(MODELO_EMBEDDING, device=device)
    encoder.max_seq_length = 128  # Limitar para velocidad

    print_gpu_usage()

    # Preparar centroides
    frases = []
    for tema, kws in TOPIC_KEYWORDS.items():
        frases.append(f"passage: {tema}. " + " ".join(kws[:5]))

    centroides = encoder.encode(
        frases,
        batch_size=32,
        show_progress_bar=False,
        convert_to_tensor=True,
        normalize_embeddings=True,
        device=device
    )

    # Procesar textos en batches grandes
    all_vecs = []
    texts_prefixed = [f"passage: {t}" for t in reps]

    t_start = time.time()
    for i in tqdm(range(0, len(texts_prefixed), BATCH_EMB), desc="Embeddings"):
        batch = texts_prefixed[i:i+BATCH_EMB]
        vecs = encoder.encode(
            batch,
            batch_size=512,  # Sub-batch interno
            show_progress_bar=False,
            convert_to_tensor=True,
            normalize_embeddings=True,
            device=device
        )
        all_vecs.append(vecs)

    # Concatenar y calcular similitudes en GPU
    all_vecs = torch.cat(all_vecs, dim=0)
    similarities = torch.matmul(all_vecs, centroides.T)
    tema_ids = similarities.argmax(dim=1).cpu().numpy()

    t_elapsed = time.time() - t_start
    print(f"   ⚡ Procesados {len(reps)} textos en {t_elapsed:.2f}s ({len(reps)/t_elapsed:.0f} textos/s)")
    print_gpu_usage()

    temas = [TOPIC_LIST[int(idx)] for idx in tema_ids]

    del encoder, centroides, all_vecs, similarities
    gc.collect()
    torch.cuda.empty_cache()

    return temas

# ===============================================================
# 9) Guardado
# ===============================================================

def guardar_resultados(df, nombre_original, grupos, t_total):
    base = re.sub(r"\.xlsx?$", "", nombre_original, flags=re.IGNORECASE)
    ts = time.strftime("%Y%m%d_%H%M%S")
    out_xlsx = f"/content/analisis_fast_{sanitize_name(base)}_{ts}.xlsx"

    n = len(df)
    n_grupos = len(grupos)

    print("\n📝 Generando Excel...")

    dist_tono = df['tono'].value_counts()
    dist_tema = df['tema'].value_counts()

    with pd.ExcelWriter(out_xlsx, engine="openpyxl") as w:
        cols = ["resumen", "tono", "tema"] + [c for c in df.columns if c not in ["resumen", "tono", "tema"]]
        df[cols].to_excel(w, sheet_name="Resultados", index=False)

        stats = pd.DataFrame({
            "Métrica": [
                "Total de textos",
                "Grupos procesados",
                "Reducción",
                "Tiempo total (s)",
                "Velocidad (textos/s)",
                "Batch Sentimiento",
                "Batch Embeddings",
                "Modelo Sentimiento",
                "Modelo Temas",
            ],
            "Valor": [
                n,
                n_grupos,
                f"{(1 - n_grupos/max(n,1))*100:.1f}%",
                f"{t_total:.2f}",
                f"{n/max(t_total,1):.0f}",
                BATCH_SENT,
                BATCH_EMB,
                MODELO_SENTIMIENTO,
                MODELO_EMBEDDING,
            ],
        })
        stats.to_excel(w, sheet_name="Estadisticas", index=False)

        dist_tono_df = pd.DataFrame({
            'Tono': dist_tono.index,
            'Cantidad': dist_tono.values,
            'Porcentaje': (dist_tono.values / n * 100).round(2)
        })
        dist_tono_df.to_excel(w, sheet_name="Distribucion_Tonos", index=False)

        dist_tema_df = pd.DataFrame({
            'Tema': dist_tema.index,
            'Cantidad': dist_tema.values,
            'Porcentaje': (dist_tema.values / n * 100).round(2)
        })
        dist_tema_df.to_excel(w, sheet_name="Distribucion_Temas", index=False)

    if os.path.exists(out_xlsx):
        print(f"💾 Descargando: {os.path.basename(out_xlsx)}")
        try:
            files.download(out_xlsx)
            print(f"✅ Descarga completada")
        except Exception as e:
            print(f"⚠️  Error: {e}")
            print(f"📁 Archivo en: {out_xlsx}")

# ===============================================================
# 10) Flujo principal
# ===============================================================

def main():
    print("="*70)
    print("  ANALIZADOR GPU-OPTIMIZED - MÁXIMA VELOCIDAD")
    print("="*70)
    print(f"⚡ Batch Sentimiento: {BATCH_SENT} textos simultáneos")
    print(f"⚡ Batch Embeddings: {BATCH_EMB} textos simultáneos")
    print(f"🔬 Modelo Sentimiento: {MODELO_SENTIMIENTO}")
    print(f"🔬 Modelo Temas: {MODELO_EMBEDDING}")
    print("="*70)

    print("\n📤 Suba su archivo Excel con columna 'resumen'")
    up = files.upload()
    if not up:
        print("❌ No se subió archivo")
        return

    nombre = list(up.keys())[0]
    print(f"\n📊 Leyendo: {nombre}")
    df = pd.read_excel(io.BytesIO(up[nombre]))

    if 'resumen' not in df.columns:
        print(f"❌ Error: Falta columna 'resumen'. Columnas: {', '.join(df.columns)}")
        return

    print(f"✓ {len(df)} textos cargados")
    textos = df['resumen'].fillna('').astype(str).map(clean_text_basic).tolist()
    nombre_excel_base = sanitize_name(re.sub(r"\.xlsx?$", "", nombre, flags=re.IGNORECASE))

    # Agrupamiento
    reps, mapa, grupos = agrupar_por_prefijo(textos)

    print("\n" + "="*70)
    print("INICIANDO PROCESAMIENTO GPU")
    print("="*70)

    t0 = time.time()

    # Sentimiento
    tonos_rep = inferir_sentimiento_gpu_fast(reps)

    # Temas
    temas_rep = inferir_temas_gpu_fast(reps)

    # Propagar
    n = len(df)
    df['tono'] = [tonos_rep[int(mapa[i])] for i in range(n)]
    df['tema'] = [temas_rep[int(mapa[i])] for i in range(n)]

    t1 = time.time()

    print(f"\n{'='*70}")
    print(f"✅ COMPLETADO")
    print(f"⚡ Tiempo total: {t1 - t0:.2f} segundos")
    print(f"⚡ Velocidad: {n/max(1e-6, t1 - t0):.0f} textos/segundo")
    print(f"{'='*70}")

    print("\n📊 DISTRIBUCIÓN:")
    print(f"\nTonos:\n{df['tono'].value_counts()}")
    print(f"\nTemas principales:\n{df['tema'].value_counts().head(5)}")

    guardar_resultados(df, nombre_excel_base, grupos, t1 - t0)

    gc.collect()
    torch.cuda.empty_cache()

    print("\n✅ Proceso completado")

if __name__ == '__main__':
    main()

✓ GPU: Tesla T4
✓ VRAM Total: 14.7 GB
✓ Compute Capability: 7.5
  ANALIZADOR GPU-OPTIMIZED - MÁXIMA VELOCIDAD
⚡ Batch Sentimiento: 2048 textos simultáneos
⚡ Batch Embeddings: 4096 textos simultáneos
🔬 Modelo Sentimiento: cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual
🔬 Modelo Temas: sentence-transformers/paraphrase-multilingual-mpnet-base-v2

📤 Suba su archivo Excel con columna 'resumen'


Saving resumen_concatenado (2).xlsx to resumen_concatenado (2) (3).xlsx

📊 Leyendo: resumen_concatenado (2) (3).xlsx
✓ 5 textos cargados
🔍 Agrupando textos similares...
✓ 5 textos → 5 grupos (0.0% reducción)

INICIANDO PROCESAMIENTO GPU

🎯 Analizando sentimiento con cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual
   Batch size: 2048 | Max length: 128
   GPU Memory: 0.54 GB allocated, 1.48 GB reserved


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Sentimiento:   0%|          | 0/1 [00:00<?, ?it/s]

   ⚡ Procesados 5 textos en 0.50s (10 textos/s)
   GPU Memory: 0.54 GB allocated, 1.48 GB reserved

🏷️  Clasificando temas con sentence-transformers/paraphrase-multilingual-mpnet-base-v2
   Batch size: 4096
   GPU Memory: 1.04 GB allocated, 1.12 GB reserved


Embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

   ⚡ Procesados 5 textos en 0.08s (63 textos/s)
   GPU Memory: 1.04 GB allocated, 1.14 GB reserved

✅ COMPLETADO
⚡ Tiempo total: 14.08 segundos
⚡ Velocidad: 0 textos/segundo

📊 DISTRIBUCIÓN:

Tonos:
tono
Neutro      4
Negativo    1
Name: count, dtype: int64

Temas principales:
tema
Salud Pública y Sanidad          3
Políticas Públicas y Gobierno    2
Name: count, dtype: int64

📝 Generando Excel...
💾 Descargando: analisis_fast_resumen_concatenado__2___3__20251021_003548.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Descarga completada

✅ Proceso completado


# ✍ Explicación

Este código es un "Analizador de Tono y Tema v13 GPU-OPTIMIZED – Máxima Velocidad 🚀" diseñado para procesar textos en español, determinar su sentimiento (positivo, negativo o neutro) y clasificarlos por tema.

### 1. Instalación e Importaciones

* `!pip install -q ...`: Instala las librerías necesarias para que el código funcione. Las más importantes son:
  * `transformers`: Permite usar modelos de lenguaje pre-entrenados (como los que analizan sentimiento).
  * `sentence-transformers`: Sirve para convertir texto en "embeddings" (representaciones numéricas que capturan el significado) y encontrar textos similares o relacionados.
  * `pandas`: Esencial para manejar datos en formato de tablas (como hojas de cálculo de Excel).
  * `openpyxl`: Necesario para leer y escribir archivos Excel (`.xlsx`).
  * `tqdm`: Muestra barras de progreso para que sepas cuánto falta en procesos largos.
  * `accelerate`: Ayuda a optimizar el uso de hardware (como la GPU) para que los modelos sean más rápidos.
* `import ...`: Carga las librerías y funciones que se usarán en el código.

### 2. Configuración

* Establece parámetros como la semilla aleatoria (`SEED`) para que los resultados sean reproducibles, detecta si tienes una GPU (tarjeta gráfica) disponible para acelerar el procesamiento y define qué modelos de lenguaje rápidos se van a usar (`MODELO_SENTIMIENTO` y `MODELO_EMBEDDING`).
* También ajusta parámetros de velocidad como el tamaño de los lotes (`BATCH_SENT`, `BATCH_EMB`) y la longitud máxima de texto a procesar (`MAX_LEN_SENT`, `MAX_CHARS_CLEAN`).

### 3. Diccionarios para temas

* Define palabras clave por tema (`TOPIC_KEYWORDS`). Estos se usan para clasificar los textos según su similitud con estos términos.

### 4. Utilidades

* Son pequeñas funciones de ayuda:
  * `sanitize_name`: Limpia nombres de archivo para que no tengan caracteres extraños.
  * `clean_text_basic`: Limpia el texto básico (quita saltos de línea, espacios extra, etc.) y lo recorta si es muy largo.
  * `normalize_for_prefix`: Prepara el texto para el agrupamiento, quitando URLs y puntuación, y pasándolo a minúsculas.
  * `print_gpu_usage`: Muestra el uso actual de la memoria de la GPU.

### 5. Agrupamiento rápido

* `agrupar_por_prefijo`: Es una función clave para la velocidad. Identifica textos que empiezan exactamente igual en una cierta longitud y los agrupa. Esto permite procesar solo un representante por cada grupo en lugar de cada texto individual, ahorrando mucho tiempo.

### 6. Sentimiento GPU-OPTIMIZADO

* Usa el modelo `cardiffnlp/twitter-xlm-roberta-base-sentiment-multilingual` para predecir el sentimiento de cada texto representante. Este modelo es **open source** y se distribuye bajo la licencia **Apache 2.0**, lo que generalmente permite su uso comercial.

### 7. Temas GPU-OPTIMIZADO

* Usa el modelo `sentence-transformers/paraphrase-multilingual-mpnet-base-v2` para convertir los textos representantes en "embeddings" (vectores numéricos). Luego compara estos vectores con los vectores promedio de las palabras clave de cada tema para encontrar el tema más similar. Este modelo también es **open source** y se basa en modelos con licencia **Apache 2.0**, permitiendo su uso comercial.

### 8. Guardado

* `guardar_resultados`: Toma los resultados del análisis (sentimiento y tema para cada texto), los añade al DataFrame original y guarda todo en un archivo Excel. También genera hojas con estadísticas del procesamiento y distribuciones de tonos y temas.

### 9. Flujo principal (`main`)

* Esta es la función que orquesta todo.
* Te pide que subas un archivo Excel con una columna llamada "resumen".
* Limpia los textos de la columna "resumen".
* Agrupa los textos por prefijo para identificar duplicados.
* Realiza el análisis de sentimiento y tema sobre los textos representantes (los únicos de cada grupo) utilizando las funciones optimizadas para GPU.
* Propaga los resultados (sentimiento y tema) a todos los textos originales (usando el mapa de agrupamiento).
* Calcula y muestra el tiempo total que tardó el procesamiento y la velocidad.
* Muestra las distribuciones de tonos y temas encontrados.
* Guarda el archivo Excel final con los resultados del análisis y las estadísticas, y lo descarga.

En resumen, el código optimiza el análisis de grandes volúmenes de texto en español usando modelos de lenguaje rápidos y open source (con licencias que permiten uso comercial) y una técnica de agrupamiento por prefijo para evitar procesar textos idénticos o muy similares varias veces.